In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

"""Example sklearn pipeline workflow

1. Create a ColumnTranformer to preprocess data
2. Create a Pipeline to fit model
3. Fit pipeline using GridSearchCV
4. Save best model to file
5. Read saved model to predict on new data
"""


In [3]:
import pickle

import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from category_encoders import LeaveOneOutEncoder

# Replace with whatever model import(s) you're using
from sklearn.ensemble import RandomForestClassifier

import warnings

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [4]:
%matplotlib inline

<IPython.core.display.Javascript object>

In [5]:
# Supresses scientific notation
pd.set_option("display.float_format", lambda x: "%.2f" % x)

<IPython.core.display.Javascript object>

Importing all dataset colums except for man-made metrics. Man-made metrics are subject to change and may include bias and colineraty to the model.

In [6]:
cols = [
    "position_again",
    "bf_odds",
    "venue_name",
    "date",
    "market_name",
    "condition",
    "barrier",
    "handicap_weight",
    "last_twenty_starts",
    "prize_money",
    "sex",
    "age",
    "jockey_sex",
    "days_since_last_run",
    "overall_starts",
    "overall_wins",
    "overall_places",
    "track_starts",
    "track_wins",
    "track_places",
    "firm_starts",
    "firm_wins",
    "firm_places",
    "good_starts",
    "good_wins",
    "good_places",
    "slow_starts",
    "slow_wins",
    "slow_places",
    "soft_starts",
    "soft_wins",
    "soft_places",
    "heavy_starts",
    "heavy_wins",
    "heavy_places",
    "distance_starts",
    "distance_wins",
    "distance_places",
]

<IPython.core.display.Javascript object>

In [7]:
df = pd.read_csv(
    "Data/horse_race_data.csv", usecols=cols, skipinitialspace=True, low_memory=False
)

<IPython.core.display.Javascript object>

In [9]:
pd.set_option("display.max_columns", 40)

<IPython.core.display.Javascript object>

In [10]:
df.head()

,position_again,bf_odds,venue_name,date,market_name,condition,barrier,handicap_weight,prize_money,sex,age,jockey_sex,last_twenty_starts,days_since_last_run,overall_starts,overall_wins,overall_places,track_starts,track_wins,track_places,firm_starts,firm_wins,firm_places,good_starts,good_wins,good_places,slow_starts,slow_wins,slow_places,soft_starts,soft_wins,soft_places,heavy_starts,heavy_wins,heavy_places,distance_starts,distance_wins,distance_places
0,1.00,2.88,Echuca,2016-06-27,R3 1200m Mdn,HVY9,3.00,58.50,2160.00,Gelding,3.00,Male,f3,14.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,1.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00
1,2.00,15.00,Echuca,2016-06-27,R3 1200m Mdn,HVY9,10.00,58.50,21175.00,Gelding,5.00,Male,f0x6x80x2,39.00,5.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,2.00,0.00,1.00,2.00,0.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00
2,3.00,95.00,Echuca,2016-06-27,R3 1200m Mdn,HVY9,6.00,56.50,28855.00,Mare,6.00,Male,f68477579x00,15.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,1.00,0.00,0.00,5.00,0.00,0.00,3.00,0.00,0.00,4.00,0.00,0.00
3,nan,20.00,Echuca,2016-06-27,R3 1200m Mdn,HVY9,2.00,56.50,1475.00,Filly,3.00,Male,f0,15.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
4,nan,2.74,Echuca,2016-06-27,R3 1200m Mdn,HVY9,7.00,56.50,1215.00,Filly,3.00,Male,f4x,173.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00


<IPython.core.display.Javascript object>

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241198 entries, 0 to 241197
Data columns (total 38 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   position_again       68984 non-null   float64
 1   bf_odds              241198 non-null  float64
 2   venue_name           241198 non-null  object 
 3   date                 241198 non-null  object 
 4   market_name          241198 non-null  object 
 5   condition            217218 non-null  object 
 6   barrier              214901 non-null  float64
 7   handicap_weight      214901 non-null  float64
 8   prize_money          214399 non-null  float64
 9   sex                  214399 non-null  object 
 10  age                  214399 non-null  float64
 11  jockey_sex           214399 non-null  object 
 12  last_twenty_starts   202406 non-null  object 
 13  days_since_last_run  214148 non-null  float64
 14  overall_starts       214148 non-null  float64
 15  overall_wins     

<IPython.core.display.Javascript object>

In [24]:
df.isna().mean()

position_again        0.71
bf_odds               0.00
venue_name            0.00
date                  0.00
market_name           0.00
condition             0.10
barrier               0.11
handicap_weight       0.11
prize_money           0.11
sex                   0.11
age                   0.11
jockey_sex            0.11
last_twenty_starts    0.16
days_since_last_run   0.11
overall_starts        0.11
overall_wins          0.11
overall_places        0.11
track_starts          0.11
track_wins            0.11
track_places          0.11
firm_starts           0.12
firm_wins             0.12
firm_places           0.12
good_starts           0.11
good_wins             0.11
good_places           0.11
slow_starts           0.11
slow_wins             0.11
slow_places           0.11
soft_starts           0.11
soft_wins             0.11
soft_places           0.11
heavy_starts          0.11
heavy_wins            0.11
heavy_places          0.11
distance_starts       0.11
distance_wins         0.11
d

<IPython.core.display.Javascript object>

In [23]:
df["market_name"].nunique()

4187

<IPython.core.display.Javascript object>

In [ ]:
# Fill out your column datatypes here
num_cols = ["position_again", "bf_odds", "barrier", "handicap_weight", "prize_money", "age", "days_since_last_run", "overall_starts", "overall_wins", "overall_places", "track_starts", "track_wins", "track_places", "firm_starts", "firm_wins", "firm_places", "good_starts", "good_wins", "good_places", "slow_starts", "slow_wins", "slow_places", "soft_starts", "soft_wins", "soft_places", "heavy_starts", "heavy_wins", "heavy_places", "distance_starts", "distance_wins", "distance_places"]

bin_cols = []

cat_cols = ["venue_name", "sex", "jockey_sex", ]
drop_cats = []

# figure out columns: "date", "market_name", "condition", "last_twenty_starts"

In [ ]:
# fmt: off
preprocessing = ColumnTransformer([
    # Encode categories
    # Potentially use one of the 2 below options
    ("OneHotEncoder", OneHotEncoder(drop=drop_cats), cat_cols),
    # ('leaveoneoutencoder', LeaveOneOutEncoder(), cat_cols),

    # Scale numeric columns
    # (not needed for all models but can't hurt)
    ("scaler", StandardScaler(), num_cols)
    # bin_cols we'll leave untouch
], remainder="passthrough")

In [ ]:
pipeline = Pipeline([
    # Apply ColumnTransformer steps
    ("preprocessing", preprocessing),

    # Add other steps as desired
    # ('dim_reduction', PCA()),
    # ('feature_selection', SelectKBest()),

    # Choose your model and put it here
    ("model", RandomForestClassifier()),
])

In [ ]:
grid = {
    # Use model__ with hyperprammeter names after
    "model__n_estimators": [100, 150],

    # Add other hyperparameters to tune as desired
    # 'dim_reduction__n_components': [2, 3, 4]
}
# fmt: on

In [ ]:
pipeline_cv = GridSearchCV(pipeline, grid)
pipeline_cv.fit(X_train, y_train)

print(f"Train score: {pipeline_cv.score(X_train, y_train):.4f}")
print(f"Test score: {pipeline_cv.score(X_test, y_test):.4f}")

In [ ]:
# Evaluate your model using additional appropriate metrics
# If doing regression consider:
# * Mean Absolute Error
# * Mean Absolute Percent Error
# * Fitted by Actuals scatter plot

# If doing classification consider:
# * Confusion matrix
# * Classification report

# Save to file
best_model = pipeline_cv.best_estimator_
with open("saved_model.pkl", "wb") as f:
    pickle.dump(best_model, f)


# Load from file
with open("saved_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)

preds = loaded_model.predict(new_data)